In [1]:
import torch
from torch import nn

import shutil

from compressai.zoo import load_state_dict, models

In [21]:
def load_checkpoint(arch: str, checkpoint_path: str, strict=True) -> nn.Module:
    state_dict = load_state_dict(
        torch.load(checkpoint_path, map_location=torch.device("cpu"))["state_dict"]
    )
    return models[arch].from_state_dict(state_dict, strict)

In [22]:
def save_checkpoint(state, is_best, filename):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, filename[:-8] + "_best" + filename[-8:])

In [38]:
pretrain_ckpt = "./ckpt/stf_0483.pth.tar"
save_ckpt = "./ckpt/lora_stf_0483.pth.tar"

In [39]:
model = load_checkpoint("stf", pretrain_ckpt, False)
lora_model = load_checkpoint("lora_stf", pretrain_ckpt, False)

param_dict = {n: p for n, p in model.named_parameters()}
lora_dict = lora_model.state_dict()

for n, p in lora_dict.items():
    if n.find(".conv.weight") != -1:
        lora_dict[n] = param_dict[n.replace(".conv.weight", ".weight")]
    if n.find(".conv.bias") != -1:
        lora_dict[n] = param_dict[n.replace(".conv.bias", ".bias")]

# for n in param_dict.keys():
#     print(n)

# lora_dict = {n: p for n, p in lora_dict.items() if n.find("lora") == -1}
for n in lora_dict.keys():
    print(n)


patch_embed.proj.conv.weight
patch_embed.proj.conv.bias
patch_embed.norm.weight
patch_embed.norm.bias
layers.0.blocks.0.norm1.weight
layers.0.blocks.0.norm1.bias
layers.0.blocks.0.attn.relative_position_bias_table
layers.0.blocks.0.attn.relative_position_index
layers.0.blocks.0.attn.qkv.weight
layers.0.blocks.0.attn.qkv.bias
layers.0.blocks.0.attn.proj.weight
layers.0.blocks.0.attn.proj.bias
layers.0.blocks.0.norm2.weight
layers.0.blocks.0.norm2.bias
layers.0.blocks.0.mlp.fc1.weight
layers.0.blocks.0.mlp.fc1.bias
layers.0.blocks.0.mlp.fc2.weight
layers.0.blocks.0.mlp.fc2.bias
layers.0.blocks.1.norm1.weight
layers.0.blocks.1.norm1.bias
layers.0.blocks.1.attn.relative_position_bias_table
layers.0.blocks.1.attn.relative_position_index
layers.0.blocks.1.attn.qkv.weight
layers.0.blocks.1.attn.qkv.bias
layers.0.blocks.1.attn.proj.weight
layers.0.blocks.1.attn.proj.bias
layers.0.blocks.1.norm2.weight
layers.0.blocks.1.norm2.bias
layers.0.blocks.1.mlp.fc1.weight
layers.0.blocks.1.mlp.fc1.bias


In [40]:

save_checkpoint({"state_dict":lora_dict}, False, save_ckpt)